In [1]:
%matplotlib inline

import pandas as pd
from datetime import datetime
import numpy as np
import geopandas as gp
import matplotlib.pyplot as plt
import matplotlib
from shapely.geometry import Point
from adjustText import adjust_text


# finalDatset = pd.DataFrame()

totalGroupedDevices = []


In [2]:
# load big files
import math 
import numpy as np

def generateFinalData(df_iterator):
    li = []
    # df_iterator = pd.read_csv('/Users/sathish/NCI/Final Project/Works/output8.csv',chunksize=10000000)
    #     df_iterator = pd.read_csv('/Volumes/Samsung_T5/Research Project/Datasets/Output of prescripts/Instance 1/output3.csv',chunksize=10000000)
    for index, data in enumerate(df_iterator):
        prevTotal = 0
        fileSizeList = [1200,12000,120000,240000]
        data['propagation_delay'] = data['distance']*1000/300000 
        pointIndex = data[data['Point'].isna()==False]
        print(pointIndex.shape)
        print(pointIndex.index[0])
        print(data.index[0])
        for fileSize in fileSizeList:
            chunk = 32 # in mb
            totalTransfer = math.ceil(fileSize/chunk)
            deviceNeeded = math.ceil(totalTransfer/8) # each device can hold 8 chunks
            print(totalTransfer)
            print(deviceNeeded)
            j=0
            i=pointIndex.index[0]-data.index.start
            while (i < data.shape[0] - 1) & (j < pointIndex.shape[0]-1):
                i = i+1
                if i != pointIndex.index[j]:
                    devices = 0
                    uploadBandwidth = 0
                    downloadBandwidth = 0
                    throughput = 0
                    propagationDelay = 0
                    while(devices<deviceNeeded):
                        devicesCount = data.iloc[i].devices
                        uploadBandwidthCount = data.iloc[i].avg_u_mbps
                        downloadBandwidthCount = data.iloc[i].avg_d_mbps
                        propagationDelayCount = data.iloc[i].propagation_delay
                        if(devicesCount+devices<=deviceNeeded):
                            devices += devicesCount
                            uploadBandwidth += uploadBandwidthCount*devicesCount
                            downloadBandwidth += downloadBandwidthCount*devicesCount
                            throughput += (chunk/uploadBandwidthCount)*devicesCount
                            propagationDelay += propagationDelayCount * devicesCount
                        else:
                            correctDevicesNeeded =  (devicesCount + devices) - deviceNeeded
                            devices += correctDevicesNeeded
                            uploadBandwidth += uploadBandwidthCount*correctDevicesNeeded
                            downloadBandwidth += downloadBandwidthCount*correctDevicesNeeded
                            throughput += (chunk/uploadBandwidthCount)*devicesCount
                            propagationDelay += propagationDelayCount * devicesCount
                        i=i+1
                    throughput = throughput*8 # because 8 times it will go transfering to same device (we divided by 8 on top)
                    propagationDelay = propagationDelay*8
                    k = pointIndex.index[j]
                    pointIndex.loc[k,'devices_'+str(fileSize/1200)+"GB"]= devices
                    pointIndex.loc[k,'avg_u_mbps_'+str(fileSize/1200)+"GB"]=uploadBandwidth  
                    pointIndex.loc[k,'avg_d_mbps_'+str(fileSize/1200)+"GB"]=downloadBandwidth
                    pointIndex.loc[k,'throughput_'+str(fileSize/1200)+"GB"]=throughput
                    pointIndex.loc[k,'propagation_delay_'+str(fileSize/1200)+"GB"]=propagationDelay

                    #In our result we didn't add the total time taken from user to cloud, we could get that by
                    # = throughput + propagation_delay + fileSize * 0.08 (processingdelay)

                    # the 1 at the end is queing delay assume to be one
                    pointIndex.loc[k,'system_upload_time_'+str(fileSize/1200)+"GB"] = propagationDelay + (fileSize * 0.2)/devices + 1
                    popagationDelay1000 = 1000/300000
                    # for 10 GB file in mac it took 4 mins to compress. 50 Mb per sec -> 0.02 sec per MB, We choose 4 time slower system
                    pointIndex.loc[k,'upload_time_1000_'+str(fileSize/1200)+"GB"] = popagationDelay1000 + (fileSize * 0.08) + (1 * devices) 
                    # (1 * devices) is the queuing delay, since we have multiple devices our system will not have this
                    popagationDelay500 = 500 / 300000
                    pointIndex.loc[k,'upload_time_500_'+str(fileSize/1200)+"GB"] = popagationDelay500 + (fileSize * 0.08) + (1 * devices)
                    popagationDelay250 = 250 / 300000
                    pointIndex.loc[k,'upload_time_250_'+str(fileSize/1200)+"GB"] = popagationDelay250 + (fileSize * 0.08) + (1 * devices)
                    j=j+1
                    i = pointIndex.index[j]-data.index.start
        li.append(pointIndex)
    totalDevice = pd.concat(li, axis=0, ignore_index=True)
    totalGroupedDevices.append(totalDevice)
#     print(totalGroupedDevices.head(5))

In [3]:
%%time
import pandas as pd
import glob
import os

allpath = "/volumes/Samsung_T5/Research Project/Datasets/USA/"
all_files = glob.glob(os.path.join(allpath , "*.csv"))

i =0
print(len(all_files))
for filename in all_files:
    data = pd.read_csv(filename,chunksize=10000000)
    generateFinalData(data)

7


/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:2358: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  result = fn(*args, **kwargs)
/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1599: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_c

(50, 12)
0
0
38
5
375
47
3750
469
7500
938


/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:2358: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  result = fn(*args, **kwargs)
/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1599: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_c

(40, 12)
0
0
38
5
375
47
3750
469
7500
938


...

(723, 12)
10001853
10000000
38
5


/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1599: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


375
47
3750
469
7500
938
(12, 12)
20002161
20000000
38
5
375
47
3750
469


/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1599: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


7500
938
CPU times: user 23min 48s, sys: 22.9 s, total: 24min 10s
Wall time: 24min 17s


In [5]:
# finalData = pd.DataFrame()

overAllData = pd.concat(totalGroupedDevices, axis=0, ignore_index=True)

In [6]:
overAllData.shape

(6150, 48)

In [7]:
totalGroupedDevices = overAllData

In [11]:
totalGroupedDevices = totalGroupedDevices.drop(totalGroupedDevices.columns[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]], axis=1) 

In [16]:
totalGroupedDevices.to_csv("/Users/sathish/NCI/Final Project/Works/Final Results/usa_result.csv")